In [1]:
import pandas as pd
import numpy as np
import math
import numpy as np
import pandas as pd
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

In [2]:
nltk.data.path.append('../week1/nltk_data')

In [3]:
sw = nltk.corpus.stopwords.words('english')

In [4]:
train_df = pd.read_csv('products_sentiment_train.tsv', sep='\t', header=None)
test_df = pd.read_csv('products_sentiment_test.tsv', sep='\t')

In [5]:
train_df.head()

,0,1
0,"2 . take around 10,000 640x480 pictures .",1
1,i downloaded a trial version of computer assoc...,1
2,the wrt54g plus the hga7t is a perfect solutio...,1
3,i dont especially like how music files are uns...,0
4,i was using the cheapie pail ... and it worked...,1


In [6]:
test_df.head()

,Id,text
0,0,"so , why the small digital elph , rather than ..."
1,1,3/4 way through the first disk we played on it...
2,2,better for the zen micro is outlook compatibil...
3,3,6 . play gameboy color games on it with goboy .
4,4,"likewise , i 've heard norton 2004 professiona..."


In [7]:
y_train = train_df[1].as_matrix()

In [8]:
corpus_train = train_df[0].as_matrix()

Самый лучший результат показал простой CountVectorizer с параметрами по умолчанию

In [9]:
#vectorizer = CountVectorizer(ngram_range=(3,5), analyzer='char_wb')
#ectorizer = CountVectorizer(stop_words=sw, ngram_range=(1,2))
vectorizer = CountVectorizer()
vectorizer.fit(corpus_train)

CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [10]:
X_train = vectorizer.transform(corpus_train)
X_train

<2000x3973 sparse matrix of type '<type 'numpy.int64'>'
	with 29639 stored elements in Compressed Sparse Row format>

Из всех перепробованных классификаторов самый высокий результат оказался у наивного Байесовского классификатора

In [11]:
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import RidgeClassifier

from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.naive_bayes import MultinomialNB

In [12]:
score = cross_val_score(MultinomialNB(), X_train, y_train)

In [13]:
score.mean(), score.std()

(0.7859981420701061, 0.013119205613526957)

# Тест

In [14]:
corpus_test = test_df.text.as_matrix()
X_test = vectorizer.transform(corpus_test)
X_test

<500x3973 sparse matrix of type '<type 'numpy.int64'>'
	with 7083 stored elements in Compressed Sparse Row format>

In [15]:
estimator_nb = MultinomialNB()
estimator_nb.fit(X_train, y_train)
y_pred_nb = estimator_nb.predict(X_test)

In [16]:
def save_submission_file(y):
    f = open('submission.csv', 'wb')
    f.write('Id,y\n')
    for idx,val in enumerate(y):
        f.write('{},{}\n'.format(idx,val))
    f.close()

In [17]:
#Имя команды в кагл ismirnov
#Скор на наивном байесе получился равным 0.81499
save_submission_file(y_pred_nb)